In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
# To generate GIFs
!pip install -q imageio

In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import datetime
import time
from skimage import io
from sklearn.metrics import accuracy_score

from IPython import display

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
!rm -rf ./logs/ 

## Import Data

In [0]:
!unzip real.zip

In [0]:
!unzip deepfakes.zip

In [0]:
!unzip neuraltextures.zip

In [0]:
!unzip face2face.zip

In [0]:
!unzip faceswap.zip

In [0]:
!unzip training_checkpoints.zip

In [0]:
def process_images(path, num_images):
  images = []
  for i in range(num_images):
    # Load in the images
    image=io.imread(path+'/%04d' % i+'.png')
    images.append(np.array(image))

  images=np.array(images)
  images_cropped = images[:, 30:480, 495:945, :]

  assert images_cropped.shape == (num_images, 450, 450, 3)

  images = images_cropped.reshape(images_cropped.shape[0], 450, 450, 3).astype('float32')
  images = (images - 127.5) / 127.5 # Normalize the images to [-1, 1]

  assert images.min() >= -1.0
  assert images.max() <= 1.0

  return images

In [0]:
real = process_images('real', 390)

In [0]:
BUFFER_SIZE = 390
BATCH_SIZE = 8

In [0]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(real).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Build Generator

In [0]:
#The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers to produce an image from a seed 
#(random noise). Start with a Dense layer that takes this seed as input, then upsample several times 
#until you reach the desired image size of 720, 1280, 3. Notice the tf.keras.layers.LeakyReLU activation for 
#each layer, except the output layer which uses tanh.

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(25*25*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((25, 25, 256)))
    assert model.output_shape == (None, 25, 25, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 25, 25, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(3, 3), padding='same', use_bias=False))
    assert model.output_shape == (None, 75, 75, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(3, 3), padding='same', use_bias=False))
    assert model.output_shape == (None, 225, 225, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 450, 450, 3)

    return model

In [0]:
#Use the (as yet untrained) generator to create an image.
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

In [0]:
plt.imshow(((np.array(generated_image[0, :, :, :]))*127.5 + 127.5).astype(int))
plt.show()

In [0]:
plt.imshow((real[0]*127.5 + 127.5).astype(int))
plt.show()

## Build Discriminator

In [0]:
#The discriminator is a CNN-based image classifier.
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[450, 450, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))#, activation='sigmoid'))

    return model

In [0]:
#Use the (as yet untrained) discriminator to classify the generated images as real or fake. 
#The model will be trained to output positive values for real images, and negative values for fake images.
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

In [0]:
discriminator.summary()

## Define the loss and optimizers

In [0]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

In [0]:
#This method quantifies how well the discriminator is able to distinguish real images from fakes. 
#It compares the discriminator's predictions on real images to an array of 1s, and the 
#discriminator's predictions on fake (generated) images to an array of 0s.

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss

In [0]:
#The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, 
#if the generator is performing well, the discriminator will classify the fake images as real (or 1). 
#Here, we will compare the discriminators decisions on the generated images to an array of 1s.

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
# Define our metrics
G_loss = tf.keras.metrics.Mean('gen_loss', dtype=tf.float32)
D_loss = tf.keras.metrics.Mean('disc_loss', dtype=tf.float32)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [0]:
checkpoint_dir = './training_checkpoints2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop



In [0]:
EPOCHS = 500
noise_dim = 100
num_examples_to_generate = 1

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [0]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
    
    G_loss(gen_loss)
    D_loss(disc_loss)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)
    with train_summary_writer.as_default():
      tf.summary.scalar('g_loss', G_loss.result(), step=epoch)
      tf.summary.scalar('d_loss', D_loss.result(), step=epoch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 10 epochs
    if (epoch + 1) % 10 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    G_loss.reset_states()
    D_loss.reset_states()

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**



In [0]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(6,6))

  for i in range(predictions.shape[0]):
      plt.subplot(1, 1, i+1)
      plt.imshow((np.array(predictions[i, :, :, :]) * 127.5 + 127.5).astype(int))
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Train the model

In [0]:
%tensorboard --logdir logs/gradient_tape

In [0]:
%%time
train(train_dataset, EPOCHS)

In [0]:
!zip -r /content/checkpoints.zip /content/training_checkpoints2/

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('checkpoints.zip')

Restore the latest checkpoint.

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

## Create a GIF


In [0]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [0]:
display_image(500)

Use `imageio` to create an animated gif using the images saved during training.

In [0]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

If you're working in Colab you can download the animation with the code below:

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)

## Extract the Discriminator

We extract the optimal discriminator, and use it to distinguish real and fake images.

First, load in all the fake images generated via various methods. Also generate fake images using our own generator.

In [0]:
#Load in the fake images
deepfake = process_images('deepfakes', 390)
face2face = process_images('face2face', 390)
faceswap = process_images('faceswap', 390)
neuraltextures = process_images('neuraltextures', 390)

In [0]:
generated_images = np.array(generator(tf.random.normal([10, noise_dim]), training=False))
for i in range(38):
  generated_images_next = np.array(generator(tf.random.normal([10, noise_dim]), training=False))
  generated_images = np.concatenate((generated_images, generated_images_next), axis=0)
generated_images.shape

In [0]:
def predictions(test):
  pred = []
  for i in range(test.shape[0]):
    pred.append(discriminator(test[i].reshape(1,450,450,3))[0,0])  
  return np.array(pred).flatten()

def predictions_prob(test):
  pred = []
  for i in range(test.shape[0]):
    pred.append(discriminator.predict_proba(test[i].reshape(1,450,450,3))[0,0])  
  return np.array(pred).flatten()

In [0]:
real_pred = predictions(real)
deepfake_pred = predictions(deepfake)
face2face_pred = predictions(face2face)
faceswap_pred = predictions(faceswap)
neuraltextures_pred = predictions(neuraltextures)
generated_pred = predictions(generated_images)

#real_pred_prob = predictions_prob(real)
#deepfake_pred_prob = predictions_prob(deepfake)
#face2face_pred_prob = predictions_prob(face2face)
#faceswap_pred_prob = predictions_prob(faceswap)
#neuraltextures_pred_prob = predictions_prob(neuraltextures)
#generated_pred_prob = predictions_prob(generated_images)

In [0]:
#plt.figure(figsize=(12,6))
#preds_prob=[real_pred_prob, deepfake_pred_prob, face2face_pred_prob, faceswap_pred_prob, neuraltextures_pred_prob, generated_pred_prob]
#labels=['real', 'deepfake', 'face2face', 'faceswap', 'neuraltextures', 'generated']
#for i in range(6):
#  plt.hist(preds[i], 30,label = labels[i], alpha=0.7)
#plt.title('Probabilistic Discriminator Output (separated by techniques)')
#plt.legend()
#plt.show()
#plt.savefig('pred_separated.png')

In [0]:
plt.figure(figsize=(12,6))
preds=[real_pred, deepfake_pred, face2face_pred, faceswap_pred, neuraltextures_pred, generated_pred]
labels=['real', 'deepfake', 'face2face', 'faceswap', 'neuraltextures', 'generated']
for i in range(6):
  plt.hist(preds[i], 30,label = labels[i], alpha=0.7)
plt.title('Discriminator Output (separated by techniques)')
plt.legend()
#plt.show()
plt.savefig('pred_separated500.png')

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('pred_separated500.png')

In [0]:
fig,ax=plt.subplots(1,1,figsize=(16,6))

ax.boxplot(preds)
plt.xticks([1, 2, 3, 4, 5, 6], labels)
plt.title('Discriminator Output (separated by techniques)')
#plt.show()
plt.savefig('pred_separated_box500.png')

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('pred_separated_box500.png')

In [0]:
plt.figure(figsize=(12,6))
fake_preds=np.concatenate((deepfake_pred, face2face_pred, faceswap_pred, neuraltextures_pred))#, generated_pred))
plt.hist(real_pred, 30,label = 'real', alpha=0.7, density=True)
plt.hist(fake_preds, 30,label = 'fake', alpha=0.7, density=True)
plt.title('Discriminator Output')
plt.legend()
#plt.show()
plt.savefig('pred500.png')

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('pred500.png')

In [0]:
real_pred.mean(), fake_preds.mean()

In [0]:
threshold = (real_pred.mean() + fake_preds.mean())/2
threshold

In [0]:
real_output = np.array([1 if real_pred[i]>threshold else 0 for i in range(len(real_pred))])
fake_output = np.array([1 if fake_preds[i]>threshold else 0 for i in range(len(fake_preds))])

In [0]:
y_true = np.concatenate((np.ones(len(real_pred)), np.zeros(len(fake_preds))))
y_pred = np.concatenate((real_output, fake_output))

In [0]:
accuracy_score(y_true, y_pred)

In [0]:
def epoch_tune(epoch):
  checkpoint.restore('./training_checkpoints2/ckpt-'+str(epoch))
  real_pred = predictions(real)
  deepfake_pred = predictions(deepfake)
  face2face_pred = predictions(face2face)
  faceswap_pred = predictions(faceswap)
  neuraltextures_pred = predictions(neuraltextures)
  fake_preds=np.concatenate((deepfake_pred, face2face_pred, faceswap_pred, neuraltextures_pred))
  threshold = (real_pred.mean() + fake_preds.mean())/2
  real_output = np.array([1 if real_pred[i]>threshold else 0 for i in range(len(real_pred))])
  fake_output = np.array([1 if fake_preds[i]>threshold else 0 for i in range(len(fake_preds))])
  y_true = np.concatenate((np.ones(len(real_pred)), np.zeros(len(fake_preds))))
  y_pred = np.concatenate((real_output, fake_output))

  return accuracy_score(y_true, y_pred)

In [0]:
epochs = np.linspace(1, 50, 50, dtype='int')
#diff_values = []
acc_values = []
for epoch in epochs:
  print(epoch)
  acc = epoch_tune(epoch)
  acc_values.append(acc)

In [0]:
plt.figure(figsize=(12,6))
plt.plot(np.linspace(10, 500, 50, dtype='int'), diff_values, '-o')
plt.xlabel('Epochs')
plt.ylabel('Class Mean Separation')
plt.title('Class mean separation during training')
#plt.show()
plt.savefig('meansep_train_500.png')

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('meansep_train_500.png')

In [0]:
plt.figure(figsize=(12,6))
plt.plot(np.linspace(10, 500, 50, dtype='int'), acc_values)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation accuracy during training')
#plt.show()
plt.savefig('acc_500.png')

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('acc_500.png')